<a href="https://colab.research.google.com/github/INLANA01/capstone_project_coref_with_transformers/blob/main/BERT_and_ALBERT_Masked_LM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import pandas as pd
import spacy
from spacy.symbols import PROPN, PERSON
from __future__ import unicode_literals, print_function
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from collections import defaultdict
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
text = "Osborn's Iron Patriot's activities have been increasingly jeopardized by various superheroes such as when he was tricked into attacking Tony Stark (in an early Iron Man suit) and when the New Avengers led by the new Captain America (Bucky Barnes) used Iron Patriot's tracking device on Luke Cage as a trick to destroy his own house. During his invasion of Asgard on the ``claim'' of a national security threat, Osborn's Iron Patriot armor gets dismantled remotely by Stark in the middle of Osborn's fight with Steve Rogers, revealing Osborn's Green Goblin-like facepaint in fear of the Void."

In [ ]:
print(text)

Osborn's Iron Patriot's activities have been increasingly jeopardized by various superheroes such as when he was tricked into attacking Tony Stark (in an early Iron Man suit) and when the New Avengers led by the new Captain America (Bucky Barnes) used Iron Patriot's tracking device on Luke Cage as a trick to destroy his own house. During his invasion of Asgard on the ``claim'' of a national security threat, Osborn's Iron Patriot armor gets dismantled remotely by Stark in the middle of Osborn's fight with Steve Rogers, revealing Osborn's Green Goblin-like facepaint in fear of the Void.


In [ ]:
index = 273 
pronoun = 'his'

In [ ]:
test = text[:index] + "[MASK]'s " + text[index + len(pronoun) + 1:]

In [ ]:
print(test)

Osborn's Iron Patriot's activities have been increasingly jeopardized by various superheroes such as when he was tricked into attacking Tony Stark (in an early Iron Man suit) and when the New Avengers led by the new Captain America (Bucky Barnes) used Iron Patriot's tracking device on Luke Cage as a trick to destroy his own house. During [MASK]'s invasion of Asgard on the ``claim'' of a national security threat, Osborn's Iron Patriot armor gets dismantled remotely by Stark in the middle of Osborn's fight with Steve Rogers, revealing Osborn's Green Goblin-like facepaint in fear of the Void.


In [ ]:
 ! git clone https://github.com/google-research-datasets/gap-coreference.git

Cloning into 'gap-coreference'...
remote: Enumerating objects: 19, done.
remote: Total 19 (delta 0), reused 0 (delta 0), pack-reused 19
Unpacking objects: 100% (19/19), done.


In [ ]:
train = pd.read_csv('./gap-coreference/gap-development.tsv', sep='\t')
valid = pd.read_csv('./gap-coreference/gap-validation.tsv', sep='\t')
test = pd.read_csv('./gap-coreference/gap-test.tsv', sep='\t')

In [ ]:
train.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,True,Pauline,207,False,http://en.wikipedia.org/wiki/List_of_Teachers_...
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,True,Bernard Leach,251,False,http://en.wikipedia.org/wiki/Warren_MacKenzie
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,False,De la Sota,246,True,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,False,Henry Rosenthal,336,True,http://en.wikipedia.org/wiki/Crime_(band)
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,False,Rivera,294,True,http://en.wikipedia.org/wiki/Jessica_Rivera


# Cover the pronoun using [MASK] token

In [ ]:
def createMaskedCol(dataset):
    masked_column = []
    for row in range(len(dataset)):
        text = dataset.iloc[row]['Text']
        pronoun = dataset.iloc[row]['Pronoun']
        index = dataset.iloc[row]['Pronoun-offset']
        test = text[:index] + "[MASK]'s " + text[index + len(pronoun) + 1:]
        masked_column.append(test)
    return masked_column

In [ ]:
train['masked_text'] = createMaskedCol(train)

In [ ]:
valid['masked_text'] = createMaskedCol(valid)

In [ ]:
train.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL,masked_text
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,True,Pauline,207,False,http://en.wikipedia.org/wiki/List_of_Teachers_...,Zoe Telford -- played the police officer girlf...
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,True,Bernard Leach,251,False,http://en.wikipedia.org/wiki/Warren_MacKenzie,"He grew up in Evanston, Illinois the second ol..."
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,False,De la Sota,246,True,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...,"He had been reelected to Congress, but resigne..."
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,False,Henry Rosenthal,336,True,http://en.wikipedia.org/wiki/Crime_(band),The current members of Crime have also perform...
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,False,Rivera,294,True,http://en.wikipedia.org/wiki/Jessica_Rivera,Her Santa Fe Opera debut in 2005 was as Nuria ...


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.0MB 4.4MB/s 
     |████████████████████████████████| 3.2MB 12.3MB/s 
     |████████████████████████████████| 890kB 52.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=79b6d95975819e5b1c3d960362c8760f83f3ae92deeca4b8bd78afa6731c5374
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoModel, AutoTokenizer, AutoModelWithLMHead, ElectraModel, ElectraForMaskedLM

MODEL_PATH = 'bert-base-uncased'

VOCAB = MODEL_PATH

print('== tokenizing ===')
tokenizer = BertTokenizer.from_pretrained(VOCAB)
model = BertForMaskedLM.from_pretrained(MODEL_PATH)
model.eval()

== tokenizing ===


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

# Get Top Predicted results using BERT Model 

In [ ]:
def predict(dataset):
    predicted_labels = []
    for row in range(len(dataset)):
        text = dataset.iloc[row]['masked_text']
        inputs = tokenizer.encode_plus(text, return_tensors="pt")
        tokenized_text = tokenizer.tokenize(text)
        masked_index = tokenized_text.index('[MASK]') + 1

        # print('== LM predicting ===')
        # Predict all tokens
        predictions = model(**inputs)[0]

        # confirm we were able to predict 'henson'
        predicted_index = torch.argmax(predictions[0, masked_index]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
        predicted_labels.append(predicted_token)

        if dataset.iloc[row]['A-coref']:
            print('Actual {} Predicted {}'.format( dataset.iloc[row]['A'], predicted_labels[row]))
        elif dataset.iloc[row]['B-coref']:
            print('Actual {} Predicted {}'.format( dataset.iloc[row]['B'], predicted_labels[row]))
        else:
            print('Actual {} Predicted {}'.format( 'Neither' , predicted_labels[row]))
        # print('predicted_token', predicted_token)
    return predicted_labels

In [ ]:
def results(dataset):
    for row in range(len(dataset)):
        if dataset.iloc[row]['A-coref']:
            print('Actual {} Predicted {}'.format( dataset.iloc[row]['A'], predicted_labels[row]))
        elif dataset.iloc[row]['B-coref']:
            print('Actual {} Predicted {}'.format( dataset.iloc[row]['B'], predicted_labels[row]))
        else:
            print('Actual {} Predicted {}'.format( Neither, predicted_labels[row]))

In [ ]:
 predicted_labels = predict(train)

Actual Cheryl Cassidy Predicted ['jenny']
Actual MacKenzie Predicted ['mackenzie']
Actual De la Sota Predicted ['labor']
Actual Henry Rosenthal Predicted ['rosenthal']
Actual Rivera Predicted ['rivera']
Actual Collins Predicted ['she']
Actual Neither Predicted ['asher']
Actual Robert Christgau Predicted ['critic']
Actual Kelsey Predicted ['mary']
Actual Christina Jennings Predicted ['stewart']
Actual David Onley Predicted ['twain']
Actual Eleanor Predicted ['eleanor']
Actual Williams Predicted ['moses']
Actual Kazuki Nakajima Predicted ['jarvis']
Actual Maria Predicted ['lorenzo']
Actual Neither Predicted ['she']
Actual Maurice Predicted ['maurice']
Actual Neither Predicted ['savage']
Actual Tim Predicted ['tim']
Actual Ramsey Predicted ['ramsey']
Actual Ben Predicted ['he']
Actual Kurt Predicted ['newman']
Actual Magee Predicted ['grant']
Actual McNeill Predicted ['virginia']
Actual Hartwig Predicted ['women']
Actual Martin Predicted ['men']
Actual Connie Predicted ['connie']
Actual A

In [ ]:
text = "The nurse notified the patient that, [MASK] shift would be ending in an hour."
# text = "The nurse notified the patient that, [MASK] blood would be drawn in an hour."
# text = "Osborn's Iron Patriot's activities have been increasingly jeopardized by various superheroes such as when he was tricked into attacking Tony Stark (in an early Iron Man suit) and when the New Avengers led by the new Captain America (Bucky Barnes) used Iron Patriot's tracking device on Luke Cage as a trick to destroy his own house. During [MASK]'s invasion of Asgard on the ``claim'' of a national security threat, Osborn's Iron Patriot armor gets dismantled remotely by Stark in the middle of Osborn's fight with Steve Rogers, revealing Osborn's Green Goblin-like facepaint in fear of the Void."
# text = "Jones's Iron Patriot's activities have been increasingly jeopardized by various superheroes such as when he was tricked into attacking Tony Williams (in an early Iron Man suit) and when the New Avengers led by the new Captain America (Bucky Barnes) used Iron Patriot's tracking device on Luke Cage as a trick to destroy his own house. During [MASK]'s invasion of Asgard on the ``claim'' of a national security threat, Jones's Iron Patriot armor gets dismantled remotely by Williams in the middle of Jones's fight with Steve Rogers, revealing Jones's Green Goblin-like facepaint in fear of the Void."
inputs = tokenizer.encode_plus(text, return_tensors="pt")

tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)
masked_index = tokenized_text.index('[MASK]') + 1
print(masked_index)
# print('== LM predicting ===')
# Predict all tokens
predictions = model(**inputs)[0]

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
print(predicted_token)

['the', 'nurse', 'notified', 'the', 'patient', 'that', ',', '[MASK]', 'shift', 'would', 'be', 'ending', 'in', 'an', 'hour', '.']
8
['her']


# Get Probabilities of for antecedant A and antecedant B which refer the pronoun

In [ ]:
def predict2(dataset):
    predicted_labels = []
    for row in range(len(dataset)):
        text = dataset.iloc[row]['masked_text']
        inputs = tokenizer.encode_plus(text, return_tensors="pt")

        tokenized_text = tokenizer.tokenize(text)
        masked_index = tokenized_text.index('[MASK]') + 1

        # print('== LM predicting ===')
        # Predict all tokens
        predictions = model(**inputs)[0]

        # confirm we were able to predict 'henson'
        predicted_index = torch.argmax(predictions[0, masked_index]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
        predicted_labels.append(predicted_token)

        if dataset.iloc[row]['A-coref']:
            print('Actual {} Predicted {}  Incorrect {}' .format( dataset.iloc[row]['A'], predicted_labels[row], dataset.iloc[row]['B']))
        elif dataset.iloc[row]['B-coref']:
            print('Actual {} Predicted {}  Incorrect {}'.format( dataset.iloc[row]['B'], predicted_labels[row], dataset.iloc[row]['A']))
        else:
            print('Actual {} Predicted {} Incorrect {}'.format( 'Neither' , predicted_labels[row], dataset.iloc[row]['A'] + ' & ' + dataset.iloc[row]['B']))

        top_k = 3
        probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
        top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

        for i, pred_idx in enumerate(top_k_indices):
            pred_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
            print('Token choice {} is {} '.format(i, pred_token))
            token_weight = top_k_weights[i]
            # print()

        # print('predicted_token', predicted_token)
    return predicted_labels

### Get top n prediction results

In [ ]:
top_k = 3
probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

for i, pred_idx in enumerate(top_k_indicies):
    predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
    token_weight = top_k_weights[i]

In [ ]:
 predicted_labels = predict2(valid)

Actual Neither Predicted ['china'] Incorrect Jose de Venecia Jr & Abalos
Token choice 0 is china
tensor(0.2654, grad_fn=<SelectBackward>)
Token choice 1 is mexico
tensor(0.0689, grad_fn=<SelectBackward>)
Token choice 2 is trump
tensor(0.0530, grad_fn=<SelectBackward>)
Actual Kathleen Predicted ['kathleen']  Incorrect Ellen
Token choice 0 is kathleen
tensor(0.9956, grad_fn=<SelectBackward>)
Token choice 1 is mary
tensor(0.0002, grad_fn=<SelectBackward>)
Token choice 2 is jane
tensor(0.0001, grad_fn=<SelectBackward>)
Actual Danny Predicted ['angela']  Incorrect Jason Scott Lee
Token choice 0 is angela
tensor(0.3718, grad_fn=<SelectBackward>)
Token choice 1 is danny
tensor(0.2683, grad_fn=<SelectBackward>)
Token choice 2 is nathan
tensor(0.0105, grad_fn=<SelectBackward>)
Actual Reucassel Predicted ['he']  Incorrect Debnam
Token choice 0 is he
tensor(0.9508, grad_fn=<SelectBackward>)
Token choice 1 is she
tensor(0.0320, grad_fn=<SelectBackward>)
Token choice 2 is i
tensor(0.0025, grad_fn=<

KeyboardInterrupt: ignored

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# init model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()
# init softmax to get probabilities later on
sm = torch.nn.Softmax(dim=0)
torch.set_grad_enabled(False)

# set sentence with MASK token, convert to token_ids
# sentence = "pleasant explains john's guilty conscience may lead him to confess to coakley. pleasant promises to help harry renegotiate the terms of the settlement with his ex-wife if he kills john. at his house, john denies the rumors. as john falls asleep, harry begins to suffocate [mask]'s only to be interrupted by charlotte boyd, john's hospice nurse."
sentence = "Osborn's Iron Patriot's activities have been increasingly jeopardized by various superheroes such as when he was tricked into attacking Tony Stark (in an early Iron Man suit) and when the New Avengers led by the new Captain America (Bucky Barnes) used Iron Patriot's tracking device on Luke Cage as a trick to destroy his own house. During [MASK]'s invasion of Asgard on the ``claim'' of a national security threat, Osborn's Iron Patriot armor gets dismantled remotely by Stark in the middle of Osborn's fight with Steve Rogers, revealing Osborn's Green Goblin-like facepaint in fear of the Void."
# sentence = sentence.lower()
# sentence = f"I {tokenizer.mask_token} you"
token_ids = tokenizer.encode(sentence, return_tensors='pt')
print(token_ids)
# tensor([[ 101, 1045,  103, 2017,  102]])
# get the position of the masked token
masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero().item()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[  101, 26999,  1005,  1055,  3707, 16419,  1005,  1055,  3450,  2031,
          2042,  6233, 15333, 29477, 17080,  5422,  2011,  2536, 16251,  2229,
          2107,  2004,  2043,  2002,  2001, 24929,  2046,  7866,  4116,  9762,
          1006,  1999,  2019,  2220,  3707,  2158,  4848,  1007,  1998,  2043,
          1996,  2047, 14936,  2419,  2011,  1996,  2047,  2952,  2637,  1006,
         10131,  2100,  9957,  1007,  2109,  3707, 16419,  1005,  1055,  9651,
          5080,  2006,  5355,  7980,  2004,  1037,  7577,  2000,  6033,  2010,
          2219,  2160,  1012,  2076,   103,  1005,  1055,  5274,  1997,  2004,
         13444,  2006,  1996,  1036,  1036,  4366,  1005,  1005,  1997,  1037,
          2120,  3036,  5081,  1010, 26999,  1005,  1055,  3707, 16419,  8177,
          4152, 17293, 19512,  2011,  9762,  1999,  1996,  2690,  1997, 26999,
          1005,  1055,  2954,  2007,  3889,  7369,  1010,  8669, 26999,  1005,
          1055,  2665, 22639,  1011,  2066,  2227,  

In [ ]:
# forward
output = model(token_ids)
last_hidden_state = output[0].squeeze(0)
# only get output for masked token
# output is the size of the vocabulary
mask_hidden_state = last_hidden_state[masked_position]
# convert to probabilities (softmax)
# giving a probability for each item in the vocabulary
probs = sm(mask_hidden_state)

In [ ]:
a_id = tokenizer.convert_tokens_to_ids('osborn')
print('Osborn probability', probs[a_id].item())
# hate probability 0.008057191967964172

Osborn probability 0.8819876313209534


In [ ]:
b_id = tokenizer.convert_tokens_to_ids('stark')
print('stark probability', probs[b_id].item())
# hate probability 0.008057191967964172

stark probability 0.0519217923283577


# Get Probabilities of for antecedant A and antecedant B which refer the pronoun.
### But in this case the text is preprocessed as explained in 2.2 Name Replacement in https://www.aclweb.org/anthology/W19-3811.pdf

In [ ]:
def createLowerCol(dataset):
    lower = []
    for row in range(len(valid)):
        # line.replace(text_to_search, replacement_text)
        # print(valid.iloc[row]['masked_text'])
        text = dataset.iloc[row]['masked_text']
        text = text.replace('john', 'peppy')
        text = text.replace('harry', 'hamada')
        text = text.replace(dataset.iloc[row]['A'], 'john')
        text = text.replace(dataset.iloc[row]['B'], 'harry')
        lower.append(text)
    return lower

In [ ]:
valid['lower'] = createLowerCol(valid)

In [ ]:
def predict3(dataset):
    predicted_labels = []
    actual_labels = []
    # init model and tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertForMaskedLM.from_pretrained('bert-base-uncased')
    model.eval()
    for row in range(len(dataset)):
        # sentence = dataset.iloc[row]['masked_text']
        sentence = dataset.iloc[row]['lower']

        # text = "Osborn's Iron Patriot's activities have been increasingly jeopardized by various superheroes such as when he was tricked into attacking Tony Stark (in an early Iron Man suit) and when the New Avengers led by the new Captain America (Bucky Barnes) used Iron Patriot's tracking device on Luke Cage as a trick to destroy his own house. During [MASK]'s invasion of Asgard on the ``claim'' of a national security threat, Osborn's Iron Patriot armor gets dismantled remotely by Stark in the middle of Osborn's fight with Steve Rogers, revealing Osborn's Green Goblin-like facepaint in fear of the Void."
        # text = "Jones's Iron Patriot's activities have been increasingly jeopardized by various superheroes such as when he was tricked into attacking Tony Williams (in an early Iron Man suit) and when the New Avengers led by the new Captain America (Bucky Barnes) used Iron Patriot's tracking device on Luke Cage as a trick to destroy his own house. During [MASK]'s invasion of Asgard on the ``claim'' of a national security threat, Jones's Iron Patriot armor gets dismantled remotely by Williams in the middle of Jones's fight with Steve Rogers, revealing Jones's Green Goblin-like facepaint in fear of the Void."
        sm = torch.nn.Softmax(dim=0)
        torch.set_grad_enabled(False)

        # set sentence with MASK token, convert to token_ids
        # sentence = f"Osborn's Iron Patriot's activities have been increasingly jeopardized by various superheroes such as when he was tricked into attacking Tony Stark (in an early Iron Man suit) and when the New Avengers led by the new Captain America (Bucky Barnes) used Iron Patriot's tracking device on Luke Cage as a trick to destroy his own house. During {tokenizer.mask_token}'s invasion of Asgard on the ``claim'' of a national security threat, Osborn's Iron Patriot armor gets dismantled remotely by Stark in the middle of Osborn's fight with Steve Rogers, revealing Osborn's Green Goblin-like facepaint in fear of the Void."
        #sentence = "Osborn's Iron Patriot's activities have been increasingly jeopardized by various superheroes such as when he was tricked into attacking Tony Stark (in an early Iron Man suit) and when the New Avengers led by the new Captain America (Bucky Barnes) used Iron Patriot's tracking device on Luke Cage as a trick to destroy his own house. During [MASK]'s invasion of Asgard on the ``claim'' of a national security threat, Osborn's Iron Patriot armor gets dismantled remotely by Stark in the middle of Osborn's fight with Steve Rogers, revealing Osborn's Green Goblin-like facepaint in fear of the Void."
        # sentence = f"I {tokenizer.mask_token} you"
        token_ids = tokenizer.encode(sentence, return_tensors='pt')
        # print(token_ids)
        # tensor([[ 101, 1045,  103, 2017,  102]])
        # get the position of the masked token
        masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero().item()

        # forward
        output = model(token_ids)
        last_hidden_state = output[0].squeeze(0)
        # only get output for masked token
        # output is the size of the vocabulary
        mask_hidden_state = last_hidden_state[masked_position]
        # convert to probabilities (softmax)
        # giving a probability for each item in the vocabulary
        probs = sm(mask_hidden_state)
        # A_id = tokenizer.convert_tokens_to_ids(dataset.iloc[row]['A'].lower())
        A_id = tokenizer.convert_tokens_to_ids('john')
        # print(A_id, dataset.iloc[row]['A'].lower())
        p_A = probs[A_id].item()
        print('Entity A {} Probability {}'.format(dataset.iloc[row]['A'].lower(), probs[A_id].item()))
        # B_id = tokenizer.convert_tokens_to_ids(dataset.iloc[row]['B'].lower())
        B_id = tokenizer.convert_tokens_to_ids('harry')
        p_B = probs[B_id].item()
        # print(B_id, dataset.iloc[row]['B'].lower())
        print('Entity B {} Probability {}'.format(dataset.iloc[row]['B'].lower(), probs[B_id].item()))
        print('Actual Answer :')
        if dataset.iloc[row]['A-coref']:
            print(dataset.iloc[row]['A'], 'john')
            actual_labels.append('A')
        elif dataset.iloc[row]['B-coref']:
            print(dataset.iloc[row]['B'], 'harry')
            actual_labels.append('B')
        else:
            actual_labels.append('C')
            print('Neither')
        if p_A > p_B:
            predicted_labels.append('A')
        else:
            predicted_labels.append('B')


        print()
    return actual_labels, predicted_labels

In [ ]:
 actual_labels, predicted_labels = predict3(valid)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Entity A jose de venecia jr Probability 0.09015898406505585
Entity B abalos Probability 0.005830221343785524
Actual Answer :
Neither

Entity A ellen Probability 9.842181316344067e-05
Entity B kathleen Probability 0.98052978515625
Actual Answer :
Kathleen harry

Entity A jason scott lee Probability 0.018168093636631966
Entity B danny Probability 0.2263994663953781
Actual Answer :
Danny harry

Entity A reucassel Probability 0.7928863167762756
Entity B debnam Probability 0.04058042913675308
Actual Answer :
Reucassel john

Entity A finch hatton Probability 0.700096845626831
Entity B beryl markham Probability 0.04511592164635658
Actual Answer :
Beryl Markham harry

Entity A james randi Probability 0.3615635633468628
Entity B jos* alvarez Probability 0.07537528872489929
Actual Answer :
Jos* Alvarez harry

Entity A von sanders Probability 0.002992008114233613
Entity B faik pasha Probability 0.9522435069084167
Actual Answer :
Faik Pasha harry

Entity A colin Probability 0.1884453445672989
Enti

In [ ]:
for row in range(len(valid)):
    print(valid.iloc[row]['lower'])

In [ ]:
text = ' me aigurhra iwuaefgh iuewfegh uweirhf aditya'
text = text.replace('aditya','john')
print(text)

 me aigurhra iwuaefgh iuewfegh uweirhf john


In [ ]:
count = 0
for i in range(len(predicted_labels)):
    if actual_labels[i] == predicted_labels[i]:
        count += 1
print(count * 100 / len(predicted_labels) )

63.436123348017624


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(actual_labels, predicted_labels))

              precision    recall  f1-score   support

           A       0.59      0.77      0.67       187
           B       0.69      0.70      0.70       205
           C       0.00      0.00      0.00        62

    accuracy                           0.63       454
   macro avg       0.43      0.49      0.45       454
weighted avg       0.55      0.63      0.59       454



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Get Probabilities of for antecedant A and antecedant B which refer the pronoun. Using ALBERT

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 5.8MB/s 


In [ ]:
from transformers import AlbertTokenizer, AlbertForMaskedLM
import torch

tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForMaskedLM.from_pretrained('albert-base-v2')
model.eval()

AlbertForMaskedLM(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
  

In [ ]:
from transformers import AlbertTokenizer, AlbertForMaskedLM
import torch
def predictAlbert(dataset):
    predicted_labels = []
    actual_labels = []
    tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
    model = AlbertForMaskedLM.from_pretrained('albert-base-v2')
    model.eval()
    for row in range(len(dataset)):
        # sentence = dataset.iloc[row]['masked_text']
        sentence = dataset.iloc[row]['lower']

        # init model and tokenizer

        # text = "Osborn's Iron Patriot's activities have been increasingly jeopardized by various superheroes such as when he was tricked into attacking Tony Stark (in an early Iron Man suit) and when the New Avengers led by the new Captain America (Bucky Barnes) used Iron Patriot's tracking device on Luke Cage as a trick to destroy his own house. During [MASK]'s invasion of Asgard on the ``claim'' of a national security threat, Osborn's Iron Patriot armor gets dismantled remotely by Stark in the middle of Osborn's fight with Steve Rogers, revealing Osborn's Green Goblin-like facepaint in fear of the Void."
        # text = "Jones's Iron Patriot's activities have been increasingly jeopardized by various superheroes such as when he was tricked into attacking Tony Williams (in an early Iron Man suit) and when the New Avengers led by the new Captain America (Bucky Barnes) used Iron Patriot's tracking device on Luke Cage as a trick to destroy his own house. During [MASK]'s invasion of Asgard on the ``claim'' of a national security threat, Jones's Iron Patriot armor gets dismantled remotely by Williams in the middle of Jones's fight with Steve Rogers, revealing Jones's Green Goblin-like facepaint in fear of the Void."
        sm = torch.nn.Softmax(dim=0)
        torch.set_grad_enabled(False)

        # set sentence with MASK token, convert to token_ids
        # sentence = f"Osborn's Iron Patriot's activities have been increasingly jeopardized by various superheroes such as when he was tricked into attacking Tony Stark (in an early Iron Man suit) and when the New Avengers led by the new Captain America (Bucky Barnes) used Iron Patriot's tracking device on Luke Cage as a trick to destroy his own house. During {tokenizer.mask_token}'s invasion of Asgard on the ``claim'' of a national security threat, Osborn's Iron Patriot armor gets dismantled remotely by Stark in the middle of Osborn's fight with Steve Rogers, revealing Osborn's Green Goblin-like facepaint in fear of the Void."
        #sentence = "Osborn's Iron Patriot's activities have been increasingly jeopardized by various superheroes such as when he was tricked into attacking Tony Stark (in an early Iron Man suit) and when the New Avengers led by the new Captain America (Bucky Barnes) used Iron Patriot's tracking device on Luke Cage as a trick to destroy his own house. During [MASK]'s invasion of Asgard on the ``claim'' of a national security threat, Osborn's Iron Patriot armor gets dismantled remotely by Stark in the middle of Osborn's fight with Steve Rogers, revealing Osborn's Green Goblin-like facepaint in fear of the Void."
        # sentence = f"I {tokenizer.mask_token} you"
        token_ids = tokenizer.encode(sentence, return_tensors='pt')
        # token_ids = tokenizer(sentence, return_tensors='pt')
        # print(token_ids)
        # tensor([[ 101, 1045,  103, 2017,  102]])
        # get the position of the masked token
        masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero().item()

        # forward
        output = model(token_ids)
        last_hidden_state = output[0].squeeze(0)
        # only get output for masked token
        # output is the size of the vocabulary
        mask_hidden_state = last_hidden_state[masked_position]
        # convert to probabilities (softmax)
        # giving a probability for each item in the vocabulary
        probs = sm(mask_hidden_state)
        # A_id = tokenizer.convert_tokens_to_ids(dataset.iloc[row]['A'].lower())
        A_id = tokenizer.convert_tokens_to_ids('john')
        # print(A_id, dataset.iloc[row]['A'].lower())
        p_A = probs[A_id].item()
        print('Entity A {} Probability {}'.format(dataset.iloc[row]['A'].lower(), probs[A_id].item()))
        # B_id = tokenizer.convert_tokens_to_ids(dataset.iloc[row]['B'].lower())
        B_id = tokenizer.convert_tokens_to_ids('harry')
        p_B = probs[B_id].item()
        # print(B_id, dataset.iloc[row]['B'].lower())
        print('Entity B {} Probability {}'.format(dataset.iloc[row]['B'].lower(), probs[B_id].item()))
        print('Actual Answer :')
        if dataset.iloc[row]['A-coref']:
            print(dataset.iloc[row]['A'], 'john')
            actual_labels.append('A')
        elif dataset.iloc[row]['B-coref']:
            print(dataset.iloc[row]['B'], 'harry')
            actual_labels.append('B')
        else:
            actual_labels.append('C')
            print('Neither')
        if p_A > p_B:
            predicted_labels.append('A')
        else:
            predicted_labels.append('B')


        print()
    return actual_labels, predicted_labels

In [ ]:
 actual_labels_albert, predicted_labels_albert = predictAlbert(valid)

Entity A jose de venecia jr Probability 1.644041549297981e-05
Entity B abalos Probability 9.068223153008148e-05
Actual Answer :
Neither

Entity A ellen Probability 3.5651289636007277e-06
Entity B kathleen Probability 0.00017327025125268847
Actual Answer :
Kathleen harry

Entity A jason scott lee Probability 3.59858148613057e-07
Entity B danny Probability 6.790851330151781e-05
Actual Answer :
Danny harry

Entity A reucassel Probability 9.733219485497102e-05
Entity B debnam Probability 1.3263486835057847e-05
Actual Answer :
Reucassel john

Entity A finch hatton Probability 5.994306320644682e-06
Entity B beryl markham Probability 4.664291566314205e-07
Actual Answer :
Beryl Markham harry

Entity A james randi Probability 1.957940639840672e-06
Entity B jos* alvarez Probability 0.00010608968295855448
Actual Answer :
Jos* Alvarez harry

Entity A von sanders Probability 1.2953413715877105e-05
Entity B faik pasha Probability 0.0007393528358079493
Actual Answer :
Faik Pasha harry

Entity A colin

In [ ]:
count = 0
for i in range(len(predicted_labels_albert)):
    if actual_labels_albert[i] == predicted_labels_albert[i]:
        count += 1
print(count * 100 / len(predicted_labels_albert) )

59.25110132158591


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(actual_labels_albert, predicted_labels_albert))

              precision    recall  f1-score   support

           A       0.57      0.72      0.64       187
           B       0.62      0.65      0.64       205
           C       0.00      0.00      0.00        62

    accuracy                           0.59       454
   macro avg       0.40      0.46      0.42       454
weighted avg       0.51      0.59      0.55       454



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(actual_labels_albert, predicted_labels_albert))

0.5925110132158591
